In [ ]:
# 在绘图前设置随机种子
set.seed(123)

library(tidyr)
library(dplyr)
library(ComplexHeatmap)
library(circlize) 


In [ ]:
# 定义热图颜色函数 (蓝-白-红)
col_fun <- colorRamp2(c(-8, 0, 8), c("#3498db", "#FFFFFF", "#d62728"))

# 定义注释颜色
ann_colors <- list(
  region = c(
    AMY = "#1f77b4",
    Isocortex = "#ff7f0e",
    HPF = "#009E73",
    HY = "#d62728",
    MB = "#9467bd",
    PFC = "#8c564b",
    STR = "#e377c2",
    TH = "#bcbd22",            
    Astro_Epen = "#17becf",    
    OPC_Oligo = "#7f7f7f",    
    Immune = "#ff9896",        
    Vascular = "#c5b0d5"       
  ),
  sex = c(
    Male = "#6AA221",
    Female = "#E51E78"
  )
)


In [49]:
file_path_scenic <- '/data2st1/junyi/output/atac0627/snregulation/example_go_input.csv'
file_path <- '/data1st2/junyi/output/atac0627/scenic/scplus_region_based_AUC_filterd.csv'
#file_path <- '/data1st2/junyi/output/atac0627/scenic/scplus_gene_based_AUC_filterd.csv'    
file_basename <- tools::file_path_sans_ext(basename(file_path))

In [ ]:
go_results <- read.csv(file_path)

In [ ]:
scenic_results <- read.csv(file_path_scenic)

In [ ]:
scenic_results_3r <- scenic_results %>%
  filter(sex == "Male", region %in% c("PFC","AMY","HPF"))

In [ ]:
#go_results <- read.csv('/data1st2/junyi/output/atac0627/scenic/scplus_region_based_AUC.csv')

In [ ]:
# define a R function to order the go_results based on the celltype
# funct

order_go_results <- function(go_results) {

    go_results$region <- gsub("-", "_", go_results$region)

    #先定义变量为因子
    region_order <- c("PFC", "Isocortex", "HPF", "TH","HY","MB","AMY","STR","OPC_Oligo","Astro_Epen","Immune","Vascular")
    sex_order <- c("Male", "Female")
    Neurotransmitter_order <- c("Glut", "GABA", "Chol","Dopa")

    go_results$region <- factor(go_results$region, levels = region_order)
    go_results$sex <- factor(go_results$sex, levels = sex_order)
    go_results$Neurotransmitter <- factor(go_results$Neurotransmitter, levels = Neurotransmitter_order)

    # 使用 order() 按 region、sex、Neurotransmitter 排序数据框
    go_results <- go_results[order(go_results$region, go_results$sex, go_results$Neurotransmitter,go_results$celltype), ]

    # If TF_reg in column names, then rename, use TF_reg as term_name

    if ("TF_reg" %in% colnames(go_results)) {
        go_results$term_name = go_results$TF_reg
    } else {
        go_results$term_name = go_results$TF
    }

    return(go_results)
}

go_results <- order_go_results(go_results)

In [22]:
generate_heatmap <- function(go_results) {
    # Create a heatmap、
  go_results_BP_MF = go_results
  #subset(go_results,source %in% c("GO:BP","GO:MF"))


  heatmap_data_BP_MF <- go_results_BP_MF %>%
    select(sample, term_name, nlog10_p_val_adj) %>%
    pivot_wider(names_from = term_name, values_from = nlog10_p_val_adj)


  # heatmap_data_BP_MF <- go_results_BP_MF %>%
  #   select(sample, term_name, nlog10_p_val_adj) %>%
  #   pivot_wider(names_from = term_name, values_from = nlog10_p_val_adj,values_fn = list(nlog10_p_val_adj = mean))


  mat_BP_MF <- as.data.frame(heatmap_data_BP_MF)
  rownames(mat_BP_MF) <- mat_BP_MF$sample
  mat_BP_MF$sample <- NULL
  mat_BP_MF <- as.matrix(mat_BP_MF)
  mat_BP_MF[is.na(mat_BP_MF)] <- 0
  mat_BP_MF_clipped <- pmax(pmin(mat_BP_MF, 8), -8)

  # generate the annotations for the heatmap
  annotation_row <- go_results_BP_MF %>%
    select(sample, region, sex) %>%
    distinct()
  rownames(annotation_row) <- annotation_row$sample
  annotation_row$sample <- NULL


  return (list(mat_BP_MF_clipped = mat_BP_MF_clipped, annotation_row = annotation_row))
}

go_elements <- generate_heatmap(go_results)
mat_BP_MF_clipped <- go_elements$mat_BP_MF_clipped
annotation_row <- go_elements$annotation_row


In [23]:
#write.csv(mat_BP_MF_clipped, file = "/data1st2/junyi/output/atac0627/regulon_analysis/TF_unfil_scenic_heatmap_40_40_cutoff8.csv")
write.csv(mat_BP_MF_clipped, file = paste0("/data1st2/junyi/output/atac0627/regulon_analysis/", file_basename, "_heatmap_40_40_cutoff8.csv"))

In [37]:
dev.off()  

null device 
          1

In [39]:
# 创建行注释对象 RowAnnotation
draw_heatmap <- function(mat_BP_MF_clipped, annotation_row,file_basename) {
  # 创建行注释对象
  row_ha <- rowAnnotation(
    region = annotation_row$region,
    sex = annotation_row$sex,
    col = ann_colors,
    annotation_name_side = "top"  # 注释名放在顶端
  )

  #pdf("figures/TF_scenic_heatmap_40_40_cutoff8.pdf", height = 40, width = 40) 
  pdf(paste0("/data1st2/junyi/output/atac0627/regulon_analysis/", file_basename, "_heatmap_40_40_cutoff8.pdf"), height = 40, width = 40)
  ht<-Heatmap(mat_BP_MF_clipped,
          name = "nlog10_p_val_adj",   # 颜色条标题
          col = col_fun,
          heatmap_legend_param = list(
            at = c(-8, -4, 0, 4, 8),   
            labels = c("-8", "-4", "0", "4", "8")
          ),
          show_row_names = TRUE,
          show_column_names = TRUE,
          cluster_rows = TRUE,
          cluster_columns = TRUE,
          top_annotation = NULL,
          left_annotation = row_ha,
          column_title = "Heatmap by Celltype and GO Term",
          row_names_gp = gpar(fontsize = 5),      
          column_names_gp = gpar(fontsize = 5),
          use_raster = F
          
  )
  draw(ht)
  dev.off()  
}
draw_heatmap(mat_BP_MF_clipped, annotation_row,file_basename = file_basename)

pdf 
  2

In [56]:
# only select the TF that are in the go_results and scenic_results
go_results_select <- go_results %>%
  filter(TF %in% scenic_results_3r$TF)

scenic_results_select <- scenic_results_3r %>%
  filter(TF %in% go_results_select$TF)

In [60]:
# onlyselect the term_name contains THE FULL STRING of  "_+/+"
go_results_select <- go_results_select %>%
  filter(grepl("_+/+", term_name, fixed = TRUE))


In [62]:
go_results_select <- order_go_results(go_results_select)
selected_elements <- generate_heatmap(go_results_select)
selected_BP_MF_clipped <- selected_elements$mat_BP_MF_clipped
selected_annotation_row <- selected_elements$annotation_row
#write.csv(mat_BP_MF_clipped, file = "/data1st2/junyi/output/atac0627/regulon_analysis/TF_scenic_heatmap_40_40_cutoff8.csv")
write.csv(selected_BP_MF_clipped, file = paste0("/data1st2/junyi/output/atac0627/regulon_analysis/", file_basename, "_selected_heatmap_40_40_cutoff8.csv"))
draw_heatmap(selected_BP_MF_clipped, selected_annotation_row,file_basename = paste0(file_basename, "_selected"))

pdf 
  2

In [ ]:
# As to your second questions. The + and - signs refer to the TF-to-gene and region-to-gene correlation coefficients. The first sign indicates wether the TF expression is either positively (+) or negatively (-) correlated with the predicted target gene expression. The second sign indiciates wether the region accessibility is either positively (+) or negatively (-) correlated with the predicted target gene expression.


In [45]:
scenic_results_select <- order_go_results(scenic_results_select)
selected_elements_scenic <- generate_heatmap(scenic_results_select)
selected_BP_MF_clipped_scenic <- selected_elements_scenic$mat_BP_MF_clipped
selected_annotation_row_scenic <- selected_elements_scenic$annotation_row
#write.csv(mat_BP_MF_clipped, file = "/data1st2/junyi/output/atac0627/regulon_analysis/TF_scenic_heatmap_40_40_cutoff8.csv")
write.csv(selected_BP_MF_clipped_scenic, file = paste0("/data1st2/junyi/output/atac0627/regulon_analysis/", file_basename, "_selected_scenic_heatmap_40_40_cutoff8.csv"))
draw_heatmap(selected_BP_MF_clipped_scenic, selected_annotation_row_scenic,file_basename = paste0(file_basename, "_selected_scenic"))

pdf 
  2

In [63]:
list_inter<-c('Zfp207',
 'Fosb',
 'Cux1',
 'Atf6',
 'Junb',
 'Tcf12',
 'Mta3',
 'Mef2a',
 'Etv5',
 'Xbp1',
 'Meis1',
 'Nfix',
 'Tfdp2',
 'Tcf4',
 'Max',
 'Bach2',
 'Gtf2ird1')

In [ ]:
scenic_results_select$region <- gsub("-", "_", scenic_results_select$region)

#先定义变量为因子
region_order <- c("PFC", "Isocortex", "HPF", "TH","HY","MB","AMY","STR","OPC_Oligo","Astro_Epen","Immune","Vascular")
sex_order <- c("Male", "Female")
table(scenic_results_select$Neurotransmitter)
Neurotransmitter_order <- c("Glut", "GABA", "Chol","Dopa")

scenic_results_select$region <- factor(scenic_results_select$region, levels = region_order)
scenic_results_select$sex <- factor(scenic_results_select$sex, levels = sex_order)
scenic_results_select$Neurotransmitter <- factor(scenic_results_select$Neurotransmitter, levels = Neurotransmitter_order)

# 使用 order() 按 region、sex、Neurotransmitter 排序数据框
scenic_results_select <- scenic_results_select[order(scenic_results_select$region, scenic_results_select$sex, scenic_results_select$Neurotransmitter, scenic_results_select$celltype), ]


In [ ]:
if ("TF_reg" %in% colnames(go_results)) {
    scenic_results_select$term_name = scenic_results_select$TF_reg
} else {
    scenic_results_select$term_name = scenic_results_select$TF
}

In [ ]:



##########cutoff10




mat_BP_MF_clipped <- pmax(pmin(mat_BP_MF, 10), -10)

annotation_row <- go_results_BP_MF %>%
  select(sample, region, sex) %>%
  distinct()

rownames(annotation_row) <- annotation_row$sample
annotation_row$sample <- NULL

#定义热图颜色函数 (蓝-白-红)
col_fun <- colorRamp2(c(-10, 0, 10), c("#3498db", "#FFFFFF", "#d62728"))

#定义注释颜色
ann_colors <- list(
  region = c(
    AMY = "#1f77b4",
    Isocortex = "#ff7f0e",
    HPF = "#009E73",
    HY = "#d62728",
    MB = "#9467bd",
    PFC = "#8c564b",
    STR = "#e377c2",
    TH = "#bcbd22",            
    Astro_Epen = "#17becf",    
    OPC_Oligo = "#7f7f7f",     
    Immune = "#ff9896",        
    Vascular = "#c5b0d5"       
  ),
  sex = c(
    Male = "#6AA221",
    Female = "#E51E78"
  )
)

#创建行注释对象 RowAnnotation
row_ha <- rowAnnotation(
  region = annotation_row$region,
  sex = annotation_row$sex,
  col = ann_colors,
  annotation_name_side = "top"  # 注释名放在顶端
)


pdf("GO_BP_MF_heatmap_40_40_cutoff10.pdf", height = 40, width = 40)  
Heatmap(mat_BP_MF_clipped,
        name = "-log10(p_val_adj)",  
        col = col_fun,
        heatmap_legend_param = list(
          at = c(-10, -5, 0, 5, 10),   
          labels = c("-10", "-5", "0", "5", "10")
        ),
        show_row_names = TRUE,
        show_column_names = TRUE,
        cluster_rows = TRUE,
        cluster_columns = TRUE,
        top_annotation = NULL,
        left_annotation = row_ha,
        column_title = "Heatmap by Celltype and GO Term",
        row_names_gp = gpar(fontsize = 5),       
        column_names_gp = gpar(fontsize = 1),
        use_raster = F
        
)
dev.off()  

##########cutoff8
#################v2 

mat_BP_MF_clipped <- pmax(pmin(mat_BP_MF, 8), -8)


set.seed(111)

annotation_row <- go_results_BP_MF %>%
  select(sample, region, sex) %>%
  distinct()

rownames(annotation_row) <- annotation_row$sample
annotation_row$sample <- NULL

#定义热图颜色函数 (蓝-白-红)
col_fun <- colorRamp2(c(-8, 0, 8), c("#3498db", "#FFFFFF", "#d62728"))

#定义注释颜色
ann_colors <- list(
  region = c(
    AMY = "#1f77b4",
    Isocortex = "#ff7f0e",
    HPF = "#009E73",
    HY = "#d62728",
    MB = "#9467bd",
    PFC = "#8c564b",
    STR = "#e377c2",
    TH = "#bcbd22",            
    Astro_Epen = "#17becf",    
    OPC_Oligo = "#7f7f7f",     
    Immune = "#ff9896",        
    Vascular = "#c5b0d5"       
  ),
  sex = c(
    Male = "#6AA221",
    Female = "#E51E78"
  )
)

#创建行注释对象 RowAnnotation
row_ha <- rowAnnotation(
  region = annotation_row$region,
  sex = annotation_row$sex,
  col = ann_colors,
  annotation_name_side = "top"  # 注释名放在顶端
)

# 转置矩阵
mat_t <- t(mat_BP_MF_clipped)

#转置注释
annotation_col <- annotation_row
rownames(annotation_col) <- rownames(annotation_row)

# 创建列注释（原来是行注释数据，转成列对应）
col_ha <- HeatmapAnnotation(
  region = annotation_col$region,
  sex = annotation_col$sex,
  col = ann_colors,
  annotation_name_side = "right"
)

pdf("GO_BP_MF_heatmap_40_40_cutoff8_rotated.pdf", height = 40, width = 40)

Heatmap(mat_t,
        name = "-log10(p_val_adj)",
        col = col_fun,
        heatmap_legend_param = list(
          at = c(-8, -4, 0, 4, 8),
          labels = c("-8", "-4", "0", "4", "8")
        ),
        show_row_names = TRUE,        # 显示旋转后行名（原来列名 GO term）
        show_column_names = TRUE,     # 显示旋转后列名（原来行名 sample）
        cluster_rows = TRUE,
        cluster_columns = TRUE,
        top_annotation = col_ha,      # 注释条放上方，且对应转置后列
        row_names_side = "right",     # 原来列名 GO term 在热图右侧
        column_names_side = "bottom", # 原来行名 sample 在热图下方
        row_names_gp = gpar(fontsize = 1),
        column_names_gp = gpar(fontsize = 5),  
        use_raster = FALSE
)

dev.off()



##########cutoff8
###############v3


# 1. 先画一次热图获取列顺序
ht_temp <- Heatmap(mat_t,
                   cluster_rows = TRUE,
                   cluster_columns = TRUE,
                   use_raster = F)

ht_temp_drawn <- draw(ht_temp)

# 2. 获取列顺序
col_order <- column_order(ht_temp_drawn)

# 3. 反转列顺序
col_order_reversed <- rev(col_order)


# 4. 使用 column_dend_reorder 参数重新绘制
pdf("GO_BP_MF_heatmap_40_40_cutoff8_rotated_reversed.pdf", height = 40, width = 40)

Heatmap(mat_t,
        name = "-log10(p_val_adj)",
        col = col_fun,
        heatmap_legend_param = list(
          at = c(-8, -4, 0, 4, 8),
          labels = c("-8", "-4", "0", "4", "8")
        ),
        show_row_names = TRUE,
        show_column_names = TRUE,
        cluster_rows = TRUE,
        cluster_columns = TRUE,
        column_dend_reorder = col_order_reversed,  
        top_annotation = col_ha,
        row_names_side = "right",
        column_names_side = "bottom",
        row_names_gp = gpar(fontsize = 1),
        column_names_gp = gpar(fontsize = 5),
        use_raster = FALSE
)

dev.off()

#################cutoff10
###############v2

mat_BP_MF_clipped <- pmax(pmin(mat_BP_MF, 10), -10)


annotation_row <- go_results_BP_MF %>%
  select(sample, region, sex) %>%
  distinct()

rownames(annotation_row) <- annotation_row$sample
annotation_row$sample <- NULL

#定义热图颜色函数 (蓝-白-红)
col_fun <- colorRamp2(c(-10, 0, 10), c("#3498db", "#FFFFFF", "#d62728"))

#定义注释颜色
ann_colors <- list(
  region = c(
    AMY = "#1f77b4",
    Isocortex = "#ff7f0e",
    HPF = "#009E73",
    HY = "#d62728",
    MB = "#9467bd",
    PFC = "#8c564b",
    STR = "#e377c2",
    TH = "#bcbd22",            
    Astro_Epen = "#17becf",    
    OPC_Oligo = "#7f7f7f",     
    Immune = "#ff9896",        
    Vascular = "#c5b0d5"       
  ),
  sex = c(
    Male = "#6AA221",
    Female = "#E51E78"
  )
)

#创建行注释对象 RowAnnotation
row_ha <- rowAnnotation(
  region = annotation_row$region,
  sex = annotation_row$sex,
  col = ann_colors,
  annotation_name_side = "top"  # 注释名放在顶端
)

#转置矩阵
mat_t <- t(mat_BP_MF_clipped)


#转置注释
annotation_col <- annotation_row
rownames(annotation_col) <- rownames(annotation_row)

# 创建列注释（原来是行注释数据，转成列对应）
col_ha <- HeatmapAnnotation(
  region = annotation_col$region,
  sex = annotation_col$sex,
  col = ann_colors,
  annotation_name_side = "right"
)

pdf("GO_BP_MF_heatmap_40_40_cutoff10_rotated.pdf", height = 40, width = 40)

Heatmap(mat_t,
        name = "-log10(p_val_adj)",
        col = col_fun,
        heatmap_legend_param = list(
          at = c(-10, -5, 0, 5, 10),
          labels = c("-10", "-5", "0", "5", "10")
        ),
        show_row_names = TRUE,        # 显示旋转后行名（原来列名 GO term）
        show_column_names = TRUE,     # 显示旋转后列名（原来行名 sample）
        cluster_rows = TRUE,
        cluster_columns = TRUE,
        top_annotation = col_ha,      # 注释条放上方，且对应转置后列
        row_names_side = "right",     # 原来列名 GO term 在热图右侧
        column_names_side = "bottom", # 原来行名 sample 在热图下方
        row_names_gp = gpar(fontsize = 1),
        column_names_gp = gpar(fontsize = 5),  
        use_raster = FALSE
)

dev.off()


#################cutoff10
###############v3


# 1. 先画一次热图获取列顺序
ht_temp <- Heatmap(mat_t,
                   cluster_rows = TRUE,
                   cluster_columns = TRUE,
                   use_raster = F)

ht_temp_drawn <- draw(ht_temp)

# 2. 获取列顺序
col_order <- column_order(ht_temp_drawn)

# 3. 反转列顺序
col_order_reversed <- rev(col_order)


# 4. 使用 column_dend_reorder 参数重新绘制
pdf("GO_BP_MF_heatmap_40_40_cutoff10_rotated_reversed.pdf", height = 40, width = 40)

Heatmap(mat_t,
        name = "-log10(p_val_adj)",
        col = col_fun,
        heatmap_legend_param = list(
          at = c(-10, -5, 0, 5, 10),
          labels = c("-10", "-5", "0", "5", "10")
        ),
        show_row_names = TRUE,
        show_column_names = TRUE,
        cluster_rows = TRUE,
        cluster_columns = TRUE,
        column_dend_reorder = col_order_reversed,  
        top_annotation = col_ha,
        row_names_side = "right",
        column_names_side = "bottom",
        row_names_gp = gpar(fontsize = 1),
        column_names_gp = gpar(fontsize = 5),
        use_raster = FALSE
)

dev.off()

